# Análise dos dados socioeconômicos e complementares dos discentes da UFRN

Primeiro iremos importar as bibliotecas/pacotes que iremos utilizar durante a execução desse notebook.

In [2]:
import re
import pandas as pd
from os import listdir
import matplotlib.pyplot as plt

Agora iremos criar uma variável chamada **dfs** que conterá os dataframes com os dados socioeconômicos e complementares dos discentes da UFRN.

In [3]:
def make_df(filenames):
    '''Pega uma lista de arquivos e retorna um único dataframe gerado.'''
    # Lista de dataframes
    dfs = [pd.read_csv('data/'+filename, sep=';') for filename in filenames]
    # Concatena todos os dataframes em um só
    return pd.concat(dfs, ignore_index=True)

# Arquivos da pasta de datasets
files = listdir('data')

# Filtro aplicado, lista de arquivos que satisfazem o regex
files_filter = list(filter(re.compile(r'dados-socio-economicos-20[0-9]{2}').search, files))

# Variável com os dataframes que iremos usar
dfs = {
    'socioeconomicos': make_df(files_filter),
    'complementares': pd.read_csv('data/dados-pessoais-discentes', sep=';'),
}

In [44]:
print(dfs['socioeconomicos'].info())
print(dfs['complementares'].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160366 entries, 0 to 160365
Data columns (total 9 columns):
id_discente                          160366 non-null object
ano                                  160366 non-null int64
periodo                              160366 non-null int64
renda                                92825 non-null float64
escola_ens_medio                     92825 non-null object
possui_bolsa_pesquisa                160366 non-null object
possui_auxilio_alimentacao           160366 non-null object
possui_auxilio_transporte            160366 non-null object
possui_auxilio_residencia_moradia    160366 non-null object
dtypes: float64(1), int64(2), object(6)
memory usage: 11.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372148 entries, 0 to 372147
Data columns (total 14 columns):
id_discente         372148 non-null object
sexo                372148 non-null object
data_nascimento     371427 non-null object
raca                372068 non-null object
es

## Análises
Iremos realizar as seguintes análises:
- Classes sociais por período de matrícula

### Classes sociais por período de matrícula
Para a elaboração da coluna **classe**, iremos utilizar a coluna de **renda** junto aos critérios de classificação econômica presente no site da [ABEP](http://www.abep.org/criterio-brasil).

In [77]:
# Limites de renda
bins = [0, 768, 1625, 2705, 4852, 9254, dfs['socioeconomicos']['renda'].max()]
# Criamos a coluna classe
dfs['socioeconomicos']['classe'] = pd.cut(
    dfs['socioeconomicos']['renda'], bins, 
    labels=["D", "C2", "C1", "B2", "B1", "A"], 
    include_lowest=True
)
# Ano periodo
dfs['socioeconomicos']['ano_periodo'] = dfs['socioeconomicos'].ano.astype(str).str.cat(
    dfs['socioeconomicos'].periodo.astype(str), sep='-'
)

In [45]:
dfs['socioeconomicos'].head()

,id_discente,ano,periodo,renda,escola_ens_medio,possui_bolsa_pesquisa,possui_auxilio_alimentacao,possui_auxilio_transporte,possui_auxilio_residencia_moradia
0,c1c7d85b65d4fb70fdbcc220e856d924,2010,2,NaN,NaN,f,f,t,f
1,37dddfeb7243a9b5efdf63e402d8d143,2010,2,NaN,NaN,f,f,f,f
2,94a39eebcaec8efcebb4b3b507231f2e,2010,2,NaN,NaN,f,f,f,f
3,e1b082d275873ab9a4a67e4e98de4384,2010,2,NaN,NaN,f,f,f,f
4,91b80d002f1596353aec4a2910fcf3aa,2010,2,NaN,NaN,f,f,f,f


In [76]:
graph_classe = dfs['socioeconomicos'].groupby(['ano', 'periodo', 'classe'])['id_discente'].count()
#.plot(kind='line', ax=ax)

# Organizando dados para exibir gráficos
graph_classe = dfs['socioeconomicos'].filter(items=['ano_periodo', 'classe', 'id_discente'])
graph_classe = graph_classe.rename(index=str,columns={'id_discente':'count'})
graph_classe = graph_classe.pivot_table(index='ano_periodo', columns='classe', aggfunc='count')

KeyError: 'classe'

In [13]:
graph_classe

count                                       
classe            D      C2      C1      B2      B1      A
ano_periodo                                               
2013-2          4.0     7.0     4.0     1.0     NaN    3.0
2014-1       2127.0  3466.0  1889.0  1442.0   850.0  746.0
2014-2       1991.0  3265.0  1718.0  1209.0   615.0  642.0
2015-1       1940.0  4035.0  1978.0  1406.0   812.0  736.0
2015-2       1573.0  4097.0  1943.0  1458.0   821.0  738.0
2016-1       1540.0  4545.0  2274.0  1523.0   842.0  829.0
2016-2       1319.0  5071.0  2928.0  2143.0  1300.0  906.0
2017-1       1347.0  5475.0  2901.0  2098.0  1091.0  986.0
2017-2       1114.0  4669.0  2519.0  1773.0   898.0  738.0
2018-1         51.0   217.0    99.0    66.0    27.0   20.0

In [18]:
graph_classe.columns

MultiIndex(levels=[['count'], ['D', 'C2', 'C1', 'B2', 'B1', 'A']],
           labels=[[0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5]],
           names=[None, 'classe'])

In [21]:
graph_classe[None]
# graph_classe.drop(['2013-2', '2018-1'], axis=1)

KeyError: <class 'object'>

In [20]:
# Limpa os dados de 2013-2 e 2018-1

# Dados dos gráficos
fig, ax = plt.subplots(figsize=(15,8))
plt.title("Classe social dos alunos por período")

# Exibir gráficos
graph_classe['count'].plot(kind='line', ax=ax)

51199

In [21]:
dfs['socioeconomicos'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160366 entries, 0 to 160365
Data columns (total 10 columns):
id_discente                          160366 non-null object
ano                                  160366 non-null int64
periodo                              160366 non-null int64
renda                                92825 non-null float64
escola_ens_medio                     92825 non-null object
possui_bolsa_pesquisa                160366 non-null object
possui_auxilio_alimentacao           160366 non-null object
possui_auxilio_transporte            160366 non-null object
possui_auxilio_residencia_moradia    160366 non-null object
classe                               92825 non-null category
dtypes: category(1), float64(1), int64(2), object(6)
memory usage: 11.2+ MB


In [5]:
dfs['complementares'].head()

,id_discente,sexo,data_nascimento,raca,estado_origem,cidade_origem,estado,municipio,bairro,nivel_ensino,forma_ingresso,ano_ingresso,periodo_ingresso,cotista
0,0e356f33d9f25c70bd7b98ef1bd1ecd4,M,03-10-1943,Não Informado,Rio Grande do Norte,NATAL,Rio Grande do Norte,NATAL,NEOPOLIS,GRADUAÇÃO,VESTIBULAR,1969,1.0,f
1,a3f7753971d7ca9c1648616b1f7e4c48,M,21-02-1949,Não Informado,Rio de Janeiro,NaN,NaN,NaN,NaN,GRADUAÇÃO,VESTIBULAR,1970,1.0,f
2,d29e0a56e3cc973750e2d2cbabedc449,F,09-09-1947,Não Informado,NaN,NaN,NaN,NaN,NaN,GRADUAÇÃO,VESTIBULAR,1970,1.0,f
3,d4f89fe5c6dd43692b3c1fc4ce324fd5,M,22-04-1949,Não Informado,Rio Grande do Norte,NaN,NaN,NaN,NaN,GRADUAÇÃO,VESTIBULAR,1970,1.0,f
4,e2f244371122aea4140427ada6f391b4,M,21-03-1944,Não Informado,Rio Grande do Norte,NATAL,Rio Grande do Norte,NATAL,ALECRIM,GRADUAÇÃO,VESTIBULAR,1970,1.0,f


In [33]:
result = pd.merge(dfs['complementares'], dfs['socioeconomicos'],  on='id_discente')

# This section will recieve date of birth, then print a table filled with ages based on joined year
ages = [] ## This will store all ages
for index,row in result.iterrows():
    if (isinstance(row['data_nascimento'],str)):
        ages.append(row['ano_ingresso'] - int(row['data_nascimento'][6:10])) ## joined year - date of birth
    else:
        ages.append(-1)  ## If element is null

## Creating table
data_raw = {'idade_ingresso': ages, 'id_discente': result['id_discente']}
ages_table = pd.DataFrame(data_raw, columns=['idade_ingresso', 'id_discente'])


# Merge Original table with ages_table
new_table = pd.merge( result, ages_table, on='id_discente')
new_table.filter(items=['idade_ingresso', 'data_nascimento', 'ano_ingresso', 'id_discente']).head()




,idade_ingresso,data_nascimento,ano_ingresso,id_discente
0,18,03-12-1983,2001,a2f469455a1226eca9d652d02ade7e4d
1,25,26-01-1977,2002,1cc536cdf93a5d423ca2f6da29c75eb5
2,22,29-11-1980,2002,ebe7eee3d3265baf1cb547335d58a1eb
3,22,29-11-1980,2002,ebe7eee3d3265baf1cb547335d58a1eb
4,22,29-11-1980,2002,ebe7eee3d3265baf1cb547335d58a1eb
